In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

In [36]:
tags = {
    "amenity": [
        "hospital",
        # "fire_station",
        # "police",
        # "college",
        # "university",
    ],
    # "shop": "mall",
    # "railway": "station"
}

# Download everything in one pass
print("downloading POIs")
pois = ox.features_from_place("Bangkok, Thailand", tags=tags).dropna(how='all', subset=["name:en"])
print(len(pois))
# Compute lat/lon of centroids
print("computing centroids")
pois = pois.to_crs(32647)
pois["y"] = pois.geometry.centroid.y
pois["x"] = pois.geometry.centroid.x


downloading POIs
107
computing centroids


/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


In [37]:
pois["geometry"] = pois.apply(lambda row: Point(row["x"], row["y"]), axis=1)

pois = pois[['geometry', 'name:en']]

In [38]:
pois_ll = pois.to_crs("EPSG:4326")

# extract columns
pois_ll["lon"] = pois_ll.geometry.x
pois_ll["lat"] = pois_ll.geometry.y

In [39]:
pois_ll

geometry  \
element id                                       
node    340033055   POINT (100.48012 13.75294)   
        1402895609   POINT (100.56167 13.8939)   
        1797189361  POINT (100.53968 13.79234)   
        1868772257  POINT (100.61817 13.90902)   
        2182981236  POINT (100.57709 13.71444)   
...                                        ...   
way     1369201067  POINT (100.55149 13.77245)   
        1396729649  POINT (100.54869 13.78879)   
        1426886300  POINT (100.78909 13.73193)   
        1434038190  POINT (100.60761 13.88214)   
        1434648444   POINT (100.5535 13.84971)   

                                                 name:en         lon  \
element id                                                             
node    340033055                      Thonburi Hospital  100.480116   
        1402895609       Mongkutwattana General Hospital  100.561668   
        1797189361               Public Health Center 11  100.539676   
        1868772257           Bhumibol Adulyadej Hospital  100.618173   
        2182981236                    Theptarin Hospital  100.577085   
...                                                  ...         ...   
way     1369201067             Veterans General Hospital  100.551491   
        1396729649                        Vimut Hospital  100.548689   
        1426886300  King Mongkut Chaokhunthahan Hospital  100.789088   
        1434038190                          SYH Hospital  100.607608   
        1434648444    Department of Corrections Hospital  100.553498   

                          lat  
element id                     
node    340033055   13.752943  
        1402895609  13.893902  
        1797189361  13.792343  
        1868772257  13.909018  
        2182981236  13.714438  
...                       ...  
way     1369201067  13.772450  
        1396729649  13.788788  
        1426886300  13.731925  
        1434038190  13.882144  
        1434648444  13.849711  

[107 rows x 4 columns]

In [40]:
pois_ll.to_csv("../processed_data/depot_locations.csv", index=False)